In [4]:
from osrsbox import monsters_api
from dash import Dash, html, dcc, callback, Output, Input
from dash.exceptions import PreventUpdate
import numpy as np
import plotly.express as px
import pandas as pd
import sys

In [3]:
monsters = monsters_api.load()
initial_monster = "zulrah"
initial_item = "tanzanite fang"

NUM_KILLS = 2000
NUM_SIMULATIONS = 1000
NUM_BINS = 100
DESIRED_RATE = 0.8

if __name__ == "__main__":
    all_db_monsters = monsters_api.load()
    # Search for the monster
    srch_monstr = next(
        (
            monster
            for monster in all_db_monsters
            if monster.name.lower() == initial_monster.lower()
        ),
        None,
    )

    if srch_monstr:
        print(f"{srch_monstr.name} found in the database.")
    else:
        print(f"{initial_monster} not found in the database.")
        sys.exit(1)

    # Search for the item
    srch_item = next(
        (
            item
            for item in srch_monstr.drops
            if item.name.lower() == f"{initial_item.lower()}"
        ),
        None,
    )

    if srch_item:
        print(f"Name: {srch_item.name}, Rarity: 1/{int(1/srch_item.rarity)}")
    else:
        print(f"{initial_item} not found in the {srch_monstr}'s database.")
        sys.exit(1)

    results = []

    for _ in range(NUM_SIMULATIONS):
        trials_to_success = np.random.geometric(srch_item.rarity, NUM_KILLS)
        results.append(trials_to_success)

    results_array = np.array(results)

    results_array_reshaped = results_array.reshape(NUM_SIMULATIONS, NUM_KILLS, 1)

    cumulative_probability = np.mean(
        results_array_reshaped <= np.arange(1, NUM_KILLS + 1), axis=1
    )

    data = {
        "Number of Kills": np.arange(1, NUM_KILLS + 1),
        "Cumulative Probability": np.mean(cumulative_probability, axis=0),
    }
    df = pd.DataFrame(data)

    filt_df = df[df["Cumulative Probability"] > DESIRED_RATE]

    if not filt_df.empty:
        kill_threshold = filt_df.iloc[0, 0]
        print(
            f"You have an {DESIRED_RATE*100}% chance of receiving your item at {kill_threshold} kills."
        )
    else:
        print(
            f"No occurrence found where cumulative probability is greater than {DESIRED_RATE} for {NUM_KILLS} kills."
        )

Zulrah found in the database.
Name: Tanzanite fang, Rarity: 1/1024
You have an 80.0% chance of receiving your item at 1649 kills.


In [3]:
fig = px.line(
    df,
    x="Number of Kills",
    y="Cumulative Probability",
    title=f"CDF of getting a {srch_item.name} from {srch_monstr.name} after {NUM_KILLS} kills",
    width=700,
    height=500,
)
fig.update_layout(title_x=0.5)
fig.show()

## Start of App Test Area  

In [67]:
app = Dash(__name__)
all_monsters = monsters_api.load()

KILLS=100
SIMULATIONS=100
RARITY=1/256

def get_all_monsters(database):
    '''Returns a list of all unique monsters and their levels.'''
    monster_options = [
        (monster.name, monster.combat_level)
        for monster in all_monsters
        if monster.name is not None
    ]
    monster_options = sorted(set(monster_options))
    monster_options = [
        {"label": f"{monster[0]} (Level {monster[1]})", "value": monster[0]}
        for monster in monster_options
    ]
    return monster_options

# Set up initial layout with 2 dropdowns and 1 graph
app.layout = html.Div(
    [
        html.H1(children="CDF for Item Drops", style={"textAlign": "center"}),
        dcc.Slider(
            0,
            10000,
            marks=None,
            value=10,
            id="kill-count",
            tooltip={"placement": "top", "always_visible": True},
        ),
        dcc.Slider(
            0,
            1000,
            marks=None,
            value=10,
            id="sim-count",
            tooltip={"placement": "bottom", "always_visible": True},
        ),
        dcc.Dropdown(
            options=get_all_monsters(all_monsters),
            value=None,
            id="text-entry",
            multi=False,
        ),
        dcc.Dropdown(value=None, id="item-dropdown", multi=False),
        dcc.Graph(id="graph-cdf"),
    ]
)

@app.callback(Output("item-dropdown", "options"), [Input("text-entry", "value")])
def update_monster_options(monster):
    if monster:
        monster_found = search_monster(monster)
        if monster_found:
            items = get_drops(monster_found)
            options = [{"label": item, "value": item} for item in items]
            return options
        else:
            return [{"label": "No items found", "value": "None"}]
    else:
        return [{"label": "Enter monster name first", "value": "None"}]


def search_monster(monster_in):
    all_db_monsters = monsters_api.load()
    search_monster = next(
        (
            monster
            for monster in all_db_monsters
            if monster.name.lower() == monster_in.lower()
        ),
        None,
    )

    if search_monster:
        return search_monster

def get_drops(monster):
    drops = [item.name for item in monster.drops]
    drops.sort()
    return drops


@app.callback(
    Output("graph-cdf", "figure"),
    [Input("text-entry", "value"), Input("item-dropdown", "value")],
)
def update_graph_content(selected_enemy, selected_item):
    # Should probably pass these two into the CDF calculation first
    if selected_enemy and selected_item:
        for _ in range(NUM_SIMULATIONS):
            trials = np.random.geometric(RARITY, NUM_KILLS)
            results.append(trials)

        results_array = np.array(results)
        results_array = results_array.reshape(NUM_SIMULATIONS, NUM_KILLS, 1)
        cumulative_probability = np.mean(
            results_array <= np.arange(1, NUM_KILLS + 1), axis=1
        )

        data = {
            "Number of Kills": np.arange(1, NUM_KILLS + 1),
            "Cumulative Probability": np.mean(cumulative_probability, axis=0),
        }
        df = pd.DataFrame(data)
        fig = px.line(
            df,
            x="Number of Kills",
            y="Cumulative Probability",
            title=f"CDF of getting the drop after {NUM_KILLS} kills",
            width=700,
            height=500,
        )
        fig.update_layout(title_x=0.5)
        return fig
    else:
        return {"layout": {"title": "Select an enemy and an item to plot the CDF"}}


if __name__ == "__main__":
    app.run(debug=True)